In [8]:
#Dash pie
from dash import Dash, html, dcc
import pandas as pd
import plotly.express as px
import plotly.io as pio
from pathlib import Path
import os

#Settings
LOCAL_CSV = r"C:\Users\whas3\Desktop\Data World\My projects\I-C\spacex_launch_dash.csv"
CLOUD_CSV = ("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/"
             "IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
OUTPUTS_DIR = Path("outputs"); OUTPUTS_DIR.mkdir(exist_ok=True)
RUN_SERVER = False           
PORT = 8050

#Load Data
try:
    df = pd.read_csv(LOCAL_CSV)
except Exception:
    df = pd.read_csv(CLOUD_CSV)

def pick(cols, *cands):
    for c in cands:
        if c in cols: return c
    raise KeyError(f"Missing columns. Tried {cands}, found {list(cols)}")

SITE_COL  = pick(df.columns, "Launch Site", "LaunchSite")
CLASS_COL = pick(df.columns, "class", "Class", "Outcome", "Success")
df[CLASS_COL] = pd.to_numeric(df[CLASS_COL], errors="coerce").fillna(0).astype(int)

#Aggregate
success_counts = (
    df[df[CLASS_COL] == 1]
      .groupby(SITE_COL)
      .size()
      .reset_index(name="Successes")
      .sort_values("Successes", ascending=False)
)

#Figure
fig = px.pie(
    success_counts,
    values="Successes",
    names=SITE_COL,
    title="Launch Success Count by Site (All Sites)",
    hole=0.25,
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.update_layout(margin=dict(l=30, r=30, t=60, b=30))

#show & save without relying on renderers 
html_file = OUTPUTS_DIR / "success_pie.html"
fig.write_html(str(html_file), include_plotlyjs="cdn", auto_open=True)  # opens in the browser
try:
    #PNG (first  pip install kaleido)
    fig.write_image(str(OUTPUTS_DIR / "success_pie.png"))
except Exception as e:
    print("[warn] Could not save PNG (install 'kaleido' to enable):", e)

#DASH APP
app = Dash(__name__)
app.title = "SpaceX Success Pie"
app.layout = html.Div(
    style={"maxWidth":"900px","margin":"40px auto","fontFamily":"system-ui, sans-serif"},
    children=[
        html.H2("Launch Success Count for All Sites"),
        dcc.Graph(id="success-pie-chart", figure=fig),
        html.Div(
            f"Data source: {'local CSV' if os.path.exists(LOCAL_CSV) else 'cloud CSV'}",
            style={"color":"#666","fontSize":"12px"},
        ),
    ],
)

if __name__ == "__main__" and RUN_SERVER:
    app.run(debug=True, port=PORT)



[warn] Could not save PNG (install 'kaleido' to enable): 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

